In [1]:
import torch
from torch import nn

## 互相关运算

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros(X.shape[0]-h+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i:i+h, j:j+w] * K)
    return Y

In [3]:
X = torch.randn(3, 3)
K = torch.randn(2, 2)
corr2d(X, K)

tensor([[-0.2457, -1.4001],
        [-0.0329,  1.0492]])

## 卷积层

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 图像中目标的边缘检测

In [7]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [10]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核

In [17]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(40):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data -= lr * conv2d.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 12.004
epoch 4, loss 2.038
epoch 6, loss 0.352
epoch 8, loss 0.063
epoch 10, loss 0.012
epoch 12, loss 0.003
epoch 14, loss 0.001
epoch 16, loss 0.000
epoch 18, loss 0.000
epoch 20, loss 0.000
epoch 22, loss 0.000
epoch 24, loss 0.000
epoch 26, loss 0.000
epoch 28, loss 0.000
epoch 30, loss 0.000
epoch 32, loss 0.000
epoch 34, loss 0.000
epoch 36, loss 0.000
epoch 38, loss 0.000
epoch 40, loss 0.000


In [18]:
conv2d.weight.data.reshape((1,2))

tensor([[ 1.0000, -1.0000]])

## 互相关和卷积

## 特征映射和感受野

## Exercises

1. 构建一个具有对角线边缘的图像X
   - 如果将本节中举例的卷积核K应用于X，会发生什么情况？
   - 如果转置X会发生什么？
   - 如果转置K会发生什么

2. 在我们创建的Conv2D自动求导时，有什么错误消息？

3. 如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法？

4. 手工设计一些卷积核
    - 二阶导数的核的形式是什么
    - 积分的核的形式是什么
    - 得到$d$次导数的最小核的大小是多少